# SpaceNet8 

SSH into a VM of your choice (I use paperspace Core machine that has ML-out-of-the-box, it has `nvidia-docker` installed).


## Download SpaceNet8 repo and build docker image

My home folder: **~/Work/SpaceNet8** (`/home/paperspace/` is a home directory `~`).

On paperspace machine get our SpaceNet github repo [instructions](https://github.com/nesaboz/SpaceNet8.git):
```
git clone https://github.com/nesaboz/SpaceNet8.git
```


Build docker image (will take a few minutes):
```
sudo nvidia-docker build -t sn8/baseline:1.0 ~/Work/SpaceNet8/docker 
```
There is a way to avoid constant `sudo` but requires messing with some json config files. For now just use `sudo`.


## Download SpaceNet8 data

Let this be folder (create via mkdir): **~/Work/data**

Install `awscli`:
```
pip install awscli
```

Download the dataset (links also [here](https://spacenet.ai/sn8-challenge/)). Try to download data first and if needed set up aws credentials:
Log into [AWS management console](https://aws.amazon.com/console/), under "Account/Security credentials/
Create access key" get ACCESS_KEY and SECRET_KEY:
```
aws configure set aws_access_key_id ACCESS_KEY  
aws configure set aws_secret_access_key SECRET_KEY
```

Download training data:
```
aws s3 cp  s3://spacenet-dataset/spacenet/SN8_floods/tarballs/Germany_Training_Public.tar.gz . 
aws s3 cp s3://spacenet-dataset/spacenet/SN8_floods/tarballs/Louisiana-East_Training_Public.tar.gz . 
```
testing data:
```
aws s3 cp  s3://spacenet-dataset/spacenet/SN8_floods/tarballs/Louisiana-West_Test_Public.tar.gz . 
```

Unzip the data:
```
tar -xf Germany_Training_Public.tar.gz
tar -xf Louisiana-East_Training_Public.tar.gz
tar -xf Louisiana-West_Test_Public.tar.gz
```



# Build and Run docker container

Let's run the container and mount the two folders that we created in previous steps:
```
sudo nvidia-docker run -v ~/Work/SpaceNet8:/tmp/SpaceNet8 -v ~/Work/data:/tmp/data -it --rm sn8/baseline:1.0 bash
```
in general, option `-v /host/path:/container/path` mounts a folder, more options [here](https://docs.docker.com/engine/reference/commandline/run/).

the prompt should now look like this `root@<container_id>:/#` and the two folders will be mounted in the `/tmp` folder.

To attach to container from VSCode, install "Remote Development" extension [pack](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.vscode-remote-extensionpack), then one can attach the container in VSCode by going to a command pallette (Cmd+Shift+P) and typing "Attach to running container".

To see running containers' info from paperspace machine use:
```
docker ps
```
To stop all the docker containers:
```
docker container stop ID_or_NAME
docker container stop $(docker container ls -aq)
```


## Data Preparation